In [ ]:
#Question 1.(b)

import csv
import numpy as np

with open('/Users/chongyeegan/Documents/Modern Analytics/Homework 1/train2.csv', 'rb') as csvfile:
    csv_file = csv.reader(csvfile, delimiter=' ', quotechar='|')
    header = csv_file.next()  

    train =[]                          
    
    for row in csv_file:      
        train.append(row)             
    train = np.array(train)
    
    print train


In [65]:

def AddBinFare(frame, fare_bracket_size=10, number_of_fares=4):
  """Bin the ticket fare and add a new column BinFare"""
  frame['BinFare'] = ((frame.Fare//fare_bracket_size)
                      .clip_upper(number_of_fares-1)
                      # Use class as substitute if no fare was given
                      .fillna(3-frame.Pclass)
                      .astype(np.int))

#in order to analyse the price collumn I need to bin up that data
#here are my binning parameters the problem we face is some of the fares are very large
#So we can either have a lot of bins with nothing in them or we can just absorb some
#information and just say anythng over 30 is just in the last bin so we add a ceiling
fare_ceiling = 40
fare_bracket_size = 10
number_of_fares = fare_ceiling // fare_bracket_size
number_of_classes = 3 #There were 1st, 2nd and 3rd classes on board

data = pd.read_csv('/Users/chongyeegan/Documents/Modern Analytics/Homework 1/train2.csv',skipinitialspace=1,index_col=[0])
AddBinFare(data,
           fare_bracket_size=fare_bracket_size,
           number_of_fares=number_of_fares)

# This reference table will show we the proportion of survivors as a function of
# Gender, class and ticket fare.
# I can now find the stats of all the women and men on board
index_list=[]
survival_list=[]
genderNames = ['female','male']
for sexIdx in range(2):
  for classIdx in range(number_of_classes):
    for fareIdx in range(number_of_fares):
      index_list += [(sexIdx,classIdx,fareIdx)]
      survival_probability = (data.Survived[(data.Sex == genderNames[sexIdx])
                                            & (data.Pclass-1==classIdx)
                                            & (data.BinFare == fareIdx)]
                              .mean())
      survival_list += [survival_probability]

# Turn into a series and transform probabilities into a binary survive label.
survival_index = pd.MultiIndex.from_tuples(index_list,
                                           names = ['Gender','Class','PriceBracket'])
survival_table = (pd.Series(survival_list,
                            index=survival_index, name='Survival')

                  # Replace nans with zeros and turn binary by assume <0.5 don't survive.
                  .fillna(0) > 0.5).astype(np.int)

print survival_table.values

# Read the test file
test = pd.read_csv('/Users/chongyeegan/Documents/Modern Analytics/Homework 1/test2.csv',index_col=[0])

# Calculate the bin fare
AddBinFare(test, fare_bracket_size=fare_bracket_size, number_of_fares=number_of_fares)

print test[['Fare','Pclass','BinFare']].head(10)

# Add survival table based on looking up the survival value of Sex,Pclass, and BinFare
test['Survived'] = (test[['Sex','Pclass','BinFare']]
                    .apply(lambda s: survival_table[(s[0]=='male', s[1]-1, s[2])],
                           axis=1)
)

# Output csv's for submission
test[['Survived']].to_csv('genderclassmodel-pandas.csv')


Gender  Class  PriceBracket
0       0      0               0
               1               0
               2               1
               3               1
        1      0               0
               1               1
               2               1
               3               1
        2      0               1
               1               1
               2               0
               3               0
1       0      0               0
               1               0
               2               0
               3               0
        1      0               0
               1               0
               2               0
               3               0
        2      0               0
               1               0
               2               0
               3               0
Name: Survival, dtype: int64


In [159]:
#With Fare Bin

import pandas as pd
import numpy as np

# Import the csv file, and skipping the first row as it's header
df = pd.read_csv('/Users/chongyeegan/Documents/Modern Analytics/Homework 1/train2.csv', header=0)

# Adding for binary values of Sex and Port of Embarkment
df['Gender'] = df['Sex'].map( {'female': 0, 'male': 1} ).astype(int) # females = 0, males = 1
#df['Port'] = df['Embarked'].map( {'C': 0, 'Q': 1, 'S':2} ).astype(int) # C = 0, Q = 1, S = 2

#Dealing with NA values of ages
median_ages = np.zeros((2,3))
for i in range(0, 2):
    for j in range(0, 3):
        median_ages[i,j] = df[(df['Gender'] == i) & (df['Pclass'] == j+1)]['Age'].dropna().median()

df['AgeFill'] = df['Age']

for i in range(0, 2):
    for j in range(0, 3):
        df.loc[ (df.Age.isnull()) & (df.Gender == i) & (df.Pclass == j+1),'AgeFill'] = median_ages[i,j]

df['AgeIsNull'] = pd.isnull(df.Age).astype(int) #annotate a passenger as 1 if its age was NA beforehand



# Converting fares into categorical features ($0-9 = 0,  $10-19 = 1,  $20-29 = 2,  $30-39 = 3) 
fare_ceiling = 40 #any fare that costs more than or equal to 40 will be put in the highest fare bin - fare bin 3
fare_bracket_size = 10
number_of_fares = fare_ceiling / fare_bracket_size
number_of_classes = 3 
df['BinFare'] = ((df.Fare/fare_bracket_size).clip_upper(number_of_fares-1).astype(np.int))

# Dealing with categorical features through dummy variables
dummy_class = pd.get_dummies(df['Pclass'], prefix='class')
dummy_embarked = pd.get_dummies(df['Embarked'], prefix='embarked')
dummy_binFare = pd.get_dummies(df['BinFare'], prefix='binFare')

cols_to_keep = (df.columns.values)
df = df[cols_to_keep].join(dummy_class.ix[:, 'class_2':]).join(dummy_embarked.ix[:, 'embarked_Q':]).join(dummy_binFare.ix[:, 'binFare_1':]) #Join dummy variables for embarked, class and bin fare
df['intercept'] = 1.0 #add intercept for categorical features


survival = df['Survived'] 

# Dropping unused features
df = df.drop(['Survived','Ticket','Cabin','Name', 'Sex','Pclass', 'Embarked', 'Age', 'Fare', 'AgeIsNull', 'BinFare'], axis=1) #ticket and cabin features have too many NA values. Name has too many categories that cannot be reduced. Sex, Age, Pclass and Embarked are ignored post data cleaning.

# Logistic Regression

y = survival
model = LogisticRegression()
model.fit(df, y)

print model.score(df, y)
 

0.805836139169


,0,1
0,PassengerId,[0.000184188765155]
1,SibSp,[-0.433983991759]
2,Parch,[-0.179509109363]
3,Gender,[-2.51086859982]
4,AgeFill,[-0.0377290687176]
5,class_2,[-0.567116890173]
6,class_3,[-1.47205698406]
7,embarked_Q,[-0.0111266228027]
8,embarked_S,[-0.340622681768]
9,binFare_1,[0.431524464703]


In [108]:
import pandas as pd
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation


# Import the csv file, and skipping the first row as it's header
train = pd.read_csv('/Users/chongyeegan/Documents/Modern Analytics/Homework 1/train2.csv', header=0)
test = pd.read_csv('/Users/chongyeegan/Documents/Modern Analytics/Homework 1/test2.csv', header=0)

print train.describe()
# Adding for binary values of Sex (females = 0, males = 1)
train['Sex'] = train['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
test['Sex'] = test['Sex'].map( {'female': 0, 'male': 1} ).astype(int)


#Dealing with NA values of ages 
median_ages = np.zeros((2,3)) #creating array of median ages for different sex and class
#mean_fare = np.zeros((2,3))
for i in range(0, 2):
    for j in range(0, 3):
        median_ages[i,j] = train[(train['Sex'] == i) & (train['Pclass'] == j+1)]['Age'].dropna().median()
        #median_fare[i,j] = train[(train['Sex'] == i) & (train['Pclass'] == j+1)]['Fare'].dropna().median()

for i in range(0, 2): #filling in the NA values with appropriate values
    for j in range(0, 3):
        train.loc[ (train.Age.isnull()) & (train.Sex == i) & (train.Pclass == j+1),'Age'] = median_ages[i,j]
        test.loc[ (test.Age.isnull()) & (test.Sex == i) & (test.Pclass == j+1),'Age'] = median_ages[i,j]
        #train.loc[ (train.Fare.isnull()) & (train.Sex == i) & (train.Pclass == j+1),'Fare'] = median_fare[i,j]
        #test.loc[ (test.Fare.isnull()) & (test.Sex == i) & (test.Pclass == j+1),'Fare'] = median_fare[i,j]

#Dealing with NA values of fare
test.loc[ (test.Fare.isnull()),'Fare'] =  train['Fare'].mean()

# Converting fares into categorical features ($0-9 = 0,  $10-19 = 1,  $20-29 = 2,  $30-39 = 3) 
fare_ceiling = 40 #any fare that costs more than or equal to 40 will be put in the highest fare bin - fare bin 3
fare_bracket_size = 10
number_of_fares = fare_ceiling / fare_bracket_size
number_of_classes = 3 
train['BinFare'] = ((train.Fare/fare_bracket_size).clip_upper(number_of_fares-1).astype(np.int))
test['BinFare'] = ((test.Fare/fare_bracket_size).clip_upper(number_of_fares-1).astype(np.int))

# Dealing with categorical features through dummy variables
dummy_class = pd.get_dummies(train['Pclass'], prefix='class')
dummy_embarked = pd.get_dummies(train['Embarked'], prefix='embarked')
dummy_binFare = pd.get_dummies(train['BinFare'], prefix='binFare')

train_cols_to_keep = (train.columns.values)
test_cols_to_keep = (test.columns.values)

train = train[train_cols_to_keep].join(dummy_class.ix[:, :]).join(dummy_embarked.ix[:, :]).join(dummy_binFare.ix[:, :]) #Join dummy variables for embarked, class and bin fare
test = test[test_cols_to_keep].join(dummy_class.ix[:, :]).join(dummy_embarked.ix[:, :]).join(dummy_binFare.ix[:, :])

train['intercept'] = 1.0 #add intercept for categorical features
test['intercept'] = 1.0 #

# Creating target vector for training data
y_train = train['Survived'] 
test['Survived'] = 0
y_test = test[ ['PassengerId', 'Survived'] ]

# Dropping unused features
train = train.drop(['PassengerId','Survived','Ticket','Cabin','Name','Pclass', 'Embarked', 'Fare'], axis=1) #ticket and cabin features have too many NA values. Name has too many categories that cannot be reduced. Sex, Age, Pclass and Embarked are ignored post data cleaning.
test = test.drop(['PassengerId','Survived','Ticket','Cabin','Name','Pclass', 'Embarked', 'Fare'], axis=1) #ticket and cabin features have too many NA values. Name has too many categories that cannot be reduced. Sex, Age, Pclass and Embarked are ignored post data cleaning.

# Creating Logistic Regression model
model = LogisticRegression()
model.fit(train, y_train)

# Finding out accuracy via cross validation score for training data
scores = cross_validation.cross_val_score(model, train, y_train)
print np.mean(scores)

#Predicting output of testing data
predicted = model.predict(test)
y_test['Survived'] = predicted

y_test.to_csv('output2.csv', index = False)




       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  
0.79797979798


In [14]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

# load dataset
dta = sm.datasets.fair.load_pandas().data

# add "affair" column: 1 represents having affairs, 0 represents not
dta['affair'] = (dta.affairs > 0).astype(int)

y, X = dmatrices('affair ~ rate_marriage + age + yrs_married + children + \
                  religious + educ + C(occupation) + C(occupation_husb)',
                  dta, return_type="dataframe")






In [92]:
#Without Categories

import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation


# Import the csv file, and skipping the first row as it's header
train = pd.read_csv('/Users/chongyeegan/Documents/Modern Analytics/Homework 1/train2.csv', header=0)
test = pd.read_csv('/Users/chongyeegan/Documents/Modern Analytics/Homework 1/test2.csv', header=0)

# Adding for binary values of Sex (females = 0, males = 1)
train['Gender'] = train['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
test['Gender'] = test['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

# Adding for binary values of Embarked (C = 0, Q = 1, S = 2)
train['Embarked'] = train['Embarked'].map( {'C': 0, 'Q': 1, 'S':2} ).astype(int)
test['Embarked'] = test['Embarked'].map({'C': 0, 'Q': 1, 'S':2}).astype(int)

#Dealing with NA values of ages 
median_ages = np.zeros((2,3)) #creating array of median ages for different gender and class
#mean_fare = np.zeros((2,3))
for i in range(0, 2):
    for j in range(0, 3):
        median_ages[i,j] = train[(train['Gender'] == i) & (train['Pclass'] == j+1)]['Age'].dropna().median()
              
train['AgeFill'] = train['Age'] #create a new feature for non-NA ages
test['AgeFill'] = test['Age']

for i in range(0, 2): #filling in the NA values with appropriate values
    for j in range(0, 3):
        train.loc[ (train.Age.isnull()) & (train.Gender == i) & (train.Pclass == j+1),'AgeFill'] = median_ages[i,j]
        test.loc[ (test.Age.isnull()) & (test.Gender == i) & (test.Pclass == j+1),'AgeFill'] = median_ages[i,j]
        #train.loc[ (train.Fare.isnull()) & (train.Gender == i) & (train.Pclass == j+1),'FareFill'] = median_fare[i,j]
        #test.loc[ (test.Fare.isnull()) & (test.Gender == i) & (test.Pclass == j+1),'FareFill'] = median_fare[i,j]

#Dealing with NA values of fare
test.loc[ (test.Fare.isnull()),'Fare'] =  train['Fare'].mean()

print train.info()

#Dealing with NA values of Embark
train.loc[ (test.Embarked.isnull()),'Embarked'] =  train['Embarked'].mean()
test.loc[ (test.Embarked.isnull()),'Embarked'] =  train['Embarked'].mean()

# Creating target vector for training data
y_train = train['Survived'] 
test['Survived'] = 0
y_test = test[ ['PassengerId', 'Survived'] ]

# Dropping unused features
train = train.drop(['PassengerId','Survived','Ticket','Cabin','Name', 'Sex', 'Age'], axis=1) #ticket and cabin features have too many NA values. Name has too many categories that cannot be reduced. Sex, Age, Pclass and Embarked are ignored post data cleaning.
test = test.drop(['PassengerId','Survived','Ticket','Cabin','Name', 'Sex', 'Age'], axis=1) #ticket and cabin features have too many NA values. Name has too many categories that cannot be reduced. Sex, Age, Pclass and Embarked are ignored post data cleaning.

# Creating Logistic Regression model
model = LogisticRegression()
model.fit(train, y_train)

# Finding out accuracy via cross validation score for training data
scores = cross_validation.cross_val_score(model, train, y_train)
print np.mean(scores)

# Predicting output of testing data
#predicted = model.predict(test)
#y_test['Survived'] = predicted

#y_test.to_csv('out_logisticRegression.csv', index = False)

train


ValueError: Cannot convert NA to integer